In [6]:
from Twitch_key import *
import socket

server = 'irc.chat.twitch.tv'
port = 6667
nickname = 'b_e_green'
token = Twitch_key()
channel = '#adinross'

In [13]:
import logging
import time
from emoji import demojize

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s — %(message)s',
                    datefmt='%Y-%m-%d_%H:%M:%S',
                    handlers=[logging.FileHandler('chat.log', encoding='utf-8')])

sock = socket.socket()
sock.connect((server, port))

sock.send(f"PASS {token}\n".encode('utf-8'))
sock.send(f"NICK {nickname}\n".encode('utf-8'))
sock.send(f"JOIN {channel}\n".encode('utf-8'))

start_time = time.time()
seconds = input("Enter: ")
seconds = int(seconds)
while True:
    current_time = time.time()
    elapsed_time = current_time - start_time

    resp = sock.recv(2048).decode('utf-8')

    if resp.startswith('PING'):
        sock.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        logging.info(demojize(resp))
        
    if elapsed_time > seconds:
        print("Finished iterating in: " + str(int(elapsed_time))  + " seconds")
        break
        sock.close()

Enter: 30
Finished iterating in: 30 seconds


In [9]:
import pandas as pd
from datetime import datetime
import re

def get_chat_dataframe(file):
    data = []

    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n\n')
        
        for line in lines:
            print(line)
            try:
                time_logged = line.split('—')[0].strip()
                time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

                username_message = line.split('—')[1:]
                username_message = '—'.join(username_message).strip()

                username, channel, message = re.search(
                    ':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message
                ).groups()

                d = {
                    'dt': time_logged,
                    'channel': channel,
                    'username': username,
                    'message': message
                }

                data.append(d)
            
            except Exception:
                pass
            
    return pd.DataFrame(data)
        

In [10]:
df = get_chat_dataframe(str(channel).replace('#',"")+'_chat.log')

In [11]:
#df.set_index('dt', inplace=True)

print(df.shape)

df.head()

(0, 0)


""
